# Johnson-Mehl tessellation picture

I'd like to draw a picture of the Johnson-Mehl tessellation and a Voronoi tessellation so the viewer can compare them.

One interesting fact which I hadn't realised until trying to draw these pictures: the boundaries between cells in the JM tessellation _aren't straight_.

There are algorithms and useful data structures for Voronoi tessellations in the [book by Okabe, Boots, Sugihara and Chiu](https://onlinelibrary.wiley.com/doi/book/10.1002/9780470317013). I've used the algorithm described by Moulinec in ["A simple and fast algorithm for computing discrete Voronoi, Johnson-Mehl or Laguerre diagrams of points"](https://www.sciencedirect.com/science/article/pii/S0965997822000618). Mainly because it's simple, although being fast is also an advantage.

## Outline
1. Sample the arrival times and locations, and prune them.
2. Assign each pixel in an image to its appropriate cell, based on which Johnson-Mehl seed grows to cover it first.
3. Compute adjacency: I want to know which cells border each other so I can colour them appropriately.
4. Colour the cells nicely. It's a planar map, so a four-colouring exists, but I think I'll be a bit simpler and use a greedy colouring, so no two neighbouring cells share a colour but maybe the number of colours isn't completely optimal.

In [32]:
import numpy as np
from scipy.spatial import KDTree
from unconstrained import sample_points, prune_arrivals

### Sampling arrival times and pruning

In [31]:
def get_arrival_times( rho, max_time=1.0, R=0 ):
    N = np.random.poisson(lam=rho*max_time*(1+2*R)**2)
    return np.sort(np.random.uniform(low=0.0, high=max_time, size=N))
# Needs redefining because of my foolishly using a global variable (rng)
# to define the version of this function in unconstrained.py.

In [30]:
rho = 100000

times = get_arrival_times(rho)
seeds = sample_points(len(times))
arrived = prune_arrivals(times, seeds)
print(f'{len(arrived)} out of {len(times)} seeds germinated.')
times = times[arrived]
seeds = seeds[arrived]

1894 out of 99891 seeds germinated.


### Assigning pixels to their cells

This is the bit using Moulinec's method. Moulinec has two separate steps: first assigning the pixels which are covered by time $T$, then the pixels which were not covered by time $T$. He chooses $T$ to optimise the speed of the algorithm. We can simplify the algorithm by choosing $T$ to be the coverage time, then there is no second step.

---

The algorithm works as follows: we start with an array $\mathcal{D}$ of "running minimum coverage times" and an array $\mathcal{I}$ of assignments, both the same shape as the output image. We intialise $\mathcal{D}$ to be full of $\infty$. We order the seeds $x_1, \dots, x_N$ with corresponding arrival times $t_1, \dots, t_N$.

Then for each $i = 1, \dots, N$ in turn: for every pixel $y$ in the ball centred at $x_i$ of radius $T-t_i$, this pixel was first reached by seed $i$ at time $\| x_i - y \| + t_i$. If $\| x_i - y \| + t_i < \mathcal{D}(y)$, then we set $\mathcal{I}(y) = i$ (overwriting its previous value if it had one) and set $\mathcal{D}(y) = \| x_i - y \| + t_i$.

Once we have done this for all $N$ seeds, every pixel is correctly assigned.

### Computing adjacency

The method is easy: for each pixel check if its cell differs from the one below and the one to the right. If they differ, then record the pair of cell IDs in the adjacency matrix. This might be a little slow, but I'm not worried about fast performance; I only plan to use this code to make pretty diagrams, so won't have more than a few hundred cells (probably fewer). As long as the resolution is high enough this will, with high probability, give us the correct adjacency structure.